# Problematic

#### For sociologists, to study the impact of stress and high-pressure environments on one’s health. They are looking for factors in people’s lives that could correlate to certain diseases or conditions. : 

They don't need to know any information about the identity of individuals. However, gender, dob and localisaiton  is really important information : 

- The id ( by name could be dropped (BE CARREFUL BECAUSE THIS INFORMATION COULD BE LINKED WITH THE SECOND PART OF THE EXERCICE)) 
- The gender could be replaced by 0 or 1 ( mais en vrai ca change pas grand chose)
- The dob : we don't need the day and the month of birth ==> Ne pourrait-on pas modifier grâce à une gaussienne de moyenne 0 sd 1 les dates de naissance ? On gagne en anonymisation, on ne perd pas grand chose 
- Zip code, we do not need the entire zipcode, we can keep only the first to digit.

==> Etant donnél'étude demandé, le risque n'est pas les identifiant mais les pseusos identifiants.

#### For the US department of Health and Human services chairman, to decide where to build new hospitals, and which departments (radiology, neurology, pulmonology, oncology...) to have in these, as the US federal government has just allocated money to build *5 new hospitals across the country* : 

A priori, je ne vois pas pourquoi on aurait besoin d'information sur les individus, ce qui nous importe est uniquement les maladies, la localicaiton, et pourquoi pas l'age des gens.

- Gender ne sert plus à rien
- l'année de naissance suffit 
- les deux premiers digit du zip code sont suffisant (étant donné que l'on va faire que 5 hopitauyx across the country)
- L'éducation on s'en branle
- Emploie on s'en fou
- Nombre d'enfant on s'en fou 
- Marital status on s'en branle
- ancestry on s'en fou 
- Nombre de véhicule ( est ce que ca peut être intéressant étant donné que c'est des hopitaux? ==> Ambulance tout ca)
- communte-time on s'en branle 
- accommodation on s'en fou 
- Maladie, pourrait on déjà regrouper les différentes maladies selon le service utilise à soigner les tiches ? On s'en branle de savoir si c'est un cancer du foie ou de la peau, ce qui est intéressant c'est qu'il faut un service d'oncologie



# Packages and data imports

## Import packages

In [4]:
#conda install -c conda-forge sklearn-pandas
import pandas as pd 
import os
import datetime as dt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.cluster import KMeans
import hashlib as h
from collections import Counter
import math


## Data import

In [5]:
data = pd.read_csv("../resources/dataset_HW3.csv")
states = pd.read_csv("../resources/states.csv")

# ID anonymisation

In [6]:
def ID_anonymised_df(df, secret_key):
    new_id_vec = []
    for id in df['id']:
        id_salted = str(id.replace(' ', '')+secret_key)
        new_id_vec.append(h.sha3_256(id_salted.encode()).hexdigest())
    
    anonimsed_df = df.copy()
    anonimsed_df['id'] = new_id_vec

    return anonimsed_df


In [7]:
data_socio = ID_anonymised_df(data, 'arthurgeeraerd11751600')
data_hospi = ID_anonymised_df(data, 'simonadrienkinartlegendre')

# Transformations 

### Date of birth

In [8]:
# Projet hopitaux : on met les ages par dizaine d'année
for i in range(len(data_hospi["dob"])):

    # Modify DOB 
    s = 0 #np.random.normal(0, 2, 1)
    data_hospi.at[i,"dob"] = str(round(int(data_hospi["dob"][i][-4:]) +s, -1))[-2:]#[0])  # ARRONDI A LA DIZAINE


# Projet sociologique : on met les ages avec une erreur suivant une normale. ++> PERMET EN PLUS DE DIFFERENCIER par rapport à l'autre BDD.

for i in range(len(data_socio["dob"])):
    # Modify DOB 
    s = np.random.normal(0, 2, 1)
    data_socio.at[i,"dob"] = round(int(data_socio["dob"][i][-4:]) +s[0]) 

## Mapping Variable catégorielles

Chaque fonction permet de gérer une variable catégorielle. On mets en entrée la valeur que l'on veut pour chaque catégorie possible, 
en retour, la database est modifiée avec les catégorie que l'on veut.

Exemple : 

change_education(data, "low", "low", "middle", "high", "high") va remplacer dans la dataBase Lessthan high school par low, high school par low, chachelor par moiddle, et master et phd par high

### functions

In [9]:
def change_education(data, less, high, bach, master, phd):

    mapping_education = {
        'Less than High School':less,
        'High School':high,
        'Bachelor':bach,
        'Masters':master,
        'PhD/md':phd,
    }
    changed_data = data.copy()
    changed_data["education"] = data.education.map(mapping_education)
    return changed_data


def change_gender(data, male, female):

    mapping_gender = {
        'male': male,
        'female': female
    }

    
    changed_data = data.copy()
    changed_data["gender"] = data.gender.map(mapping_gender)
    return changed_data


def change_employment(data, employed, retired, student, unemployed):
    mapping_employment = {
        'Employed' : employed,
        'Retired' : retired,
        'Student' : student,
        'Unemployed' : unemployed
    }

    changed_data = data.copy()
    changed_data["employment"] = data.employment.map(mapping_employment)
    return changed_data


def change_children(data, zero, un, deux, trois, quatre , cinq, six, sept, huit):
    mapping_children = {
        0 : zero,
        1 : un,
        2 : deux,
        3 : trois,
        4 : quatre,
        5 : cinq,
        6 : six,
        7 : sept,
        8 : huit
    }

    changed_data = data.copy()
    changed_data["children"] = data.children.map(mapping_children)
    return changed_data



def change_marital_status(data, divorced, married, single, widowed):
    mapping_marital_status = {
        "divorced" : divorced,
        "married" : married,
        "single" : single,
        "widowed" : widowed,

    }

    changed_data = data.copy()
    changed_data["marital_status"] = data.marital_status.map(mapping_marital_status)
    return changed_data


def change_ancestry(data, Africa, Asia, Europe, North_America, Oceania, South_America):
    mapping_ancestry = {
        "Africa" : Africa,
        "Asia" : Asia,
        "Europe" : Europe,
        "North America" : North_America,
        "Oceania" : Oceania,
        "South America" : South_America

    }

    changed_data = data.copy()
    changed_data["ancestry"] = data.ancestry.map(mapping_ancestry)
    return changed_data


def change_number_vehicles(data, zero, un, deux, trois):
    mapping_nmbr_vehicule = {
        0 : zero,
        1 : un,
        2 : deux,
        3 : trois,

    }

    changed_data = data.copy()
    changed_data["number_vehicles"] = data.number_vehicles.map(mapping_nmbr_vehicule)
    return changed_data


def change_accommodation(data, Own_flat, Own_house, Rent_flat, Rent_house, Rent_room):
    mapping_accommodation = {
        "Own flat" : Own_flat,
        "Own house" : Own_house,
        "Rent flat" : Rent_flat,
        "Rent house" : Rent_house,
        "Rent room" : Rent_room,
        
    }

    changed_data = data.copy()
    changed_data["accommodation"] = data.accommodation.map(mapping_accommodation)
    return changed_data

# for the sake of the first use case we gather all specific diseases under more general health department 

def change_disease(data, gastritis, heart_disease, hypertension, diabetes, skin_cancer, prostate_cancer, breast_cancer, schizophrenia, Alzheimer, multiple_sclerosis, endometriosis, kidney_disease, HIV_AIDS):
    mapping_disease = {
        'gastritis': gastritis ,
        'heart disease': heart_disease,
        "hypertension": hypertension,
        "diabetes": diabetes,
        "skin cancer": skin_cancer,
        "prostate cancer":  prostate_cancer,
        "breast cancer": breast_cancer, 
        "schizophrenia": schizophrenia,
        "Alzheimer's disease": Alzheimer, 
        "multiple sclerosis": multiple_sclerosis,
        "endometriosis": endometriosis, 
        "kidney disease": kidney_disease,
        "HIV/AIDS": HIV_AIDS 
    }

    changed_data = data.copy()
    changed_data['disease'] = data.disease.map(mapping_disease)
    return changed_data


    


### tests

In [10]:
# 'low' under highschool, 'high' elsewhere 
X1 = change_education(data, less= '0', high='0', bach='1', master='1', phd='1')

# '0' male, '1' female 
X2 = change_gender(X1,male = 0, female = 1)

# active = {employed, student} unactive = {retired, unemployed}
X3 = change_employment(X2, employed= '1', retired='0', student='1', unemployed='0')

# QUE FAIRE AVEC LES CHILDREN ?
X4 = change_children(X3, "none", "one",'more than one','more than one','more than one','more than one','more than one','more than one','more than one')

# married = married, unmarried = others
X5 = change_marital_status(X4, divorced='0', married='1', single='0', widowed='1')

# native : north america, foreigners : others BUT to much 'other' compare to native so might affect negatively l-diversity. Instead, we merge north/south america 
X6 = change_ancestry(X5, Africa='Africa', Asia='Asia', Europe='Europe', North_America='America', Oceania='Oceania', South_America='America')

# 1 if got vehicule, 0 else 
X7 = change_number_vehicles(X6, zero='none', un='one', deux='more than one', trois='more than one')

# owner = owner, renter = renter regardless of the type of the accomodaations (2/3 owner, maybe reset these proportion ? )
X8 = change_accommodation(X7, Own_flat='1', Own_house='1', Rent_flat= '0', Rent_house='0', Rent_room='0')

# gathered under medical departments
X9 =change_disease(X8, gastritis='gastro-enterolgy', heart_disease='cardiovascular', hypertension='cardiovascular', diabetes='endocrynology', 
    skin_cancer='oncology', prostate_cancer='oncology', breast_cancer='oncology', 
    schizophrenia='psychiatry', Alzheimer='neurology', multiple_sclerosis='neurology', endometriosis='gynecology', kidney_disease='nephrology', HIV_AIDS='interal mdeicine')

# 

## Gestion ZIPCODE

import d'un base de données trouvée sur le net : http://structnet.com/instructions/zip_min_max_by_state.html

### HOSPITAL CASE

In [16]:
dict_states = {}

for (i,state) in enumerate(states['State Name']):
    dict_states[state] = [states['Zip Min'].iloc[i,], states['Zip Max'].iloc[i,]]     

In [20]:
def change_zipcode_state(df, dic_state):
    dict_zip_to_state = {}
    new_df = df.copy()

    for i,zip in enumerate(df['zipcode']):
        found = False
        for state, r in list(dict_states.items()):
            if zip in range(int(r[0]), int(r[1])):
                dict_zip_to_state[zip] = state
                found = True
        if not found:
            dict_zip_to_state[zip] = 'unknown state'

    new_df['zipcode'] = df.zipcode.map(dict_zip_to_state)
    return new_df

On créé le dictionnaire qui pour chaque clé : 'nom_etat', fait correspondre la valuer [zip_min, zip_max]

In [22]:
test_new_data_zip = change_zipcode_state(data,dict_states)

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
0,Amelia Morrow,female,12/1950,unknown state,High School,Retired,2,married,Africa,2,0.0,Own house,Alzheimer's disease
1,Lillian Cole,female,4/1940,Texas,PhD/md,Retired,1,married,Africa,1,0.0,Rent flat,multiple sclerosis
2,Lillian Simpson,female,12/1966,Iowa,Bachelor,Employed,1,married,Asia,0,0.1,Rent flat,heart disease
3,Avery Richards,female,4/1943,Wisconsin,Bachelor,Retired,1,married,Africa,0,0.0,Rent room,endometriosis
4,Sophia Alvarado,female,10/1947,Nebraska,High School,Retired,1,married,Europe,0,0.0,Rent flat,gastritis
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Connor Pena,male,11/1997,Louisiana,High School,Student,0,divorced,Europe,0,1.1,Rent house,gastritis
1996,Caleb Bowman,male,11/1999,Michigan,High School,Student,0,single,North America,0,0.1,Rent house,gastritis
1997,Ella Barber,female,6/1948,california,Masters,Retired,5,married,Africa,1,0.0,Own house,skin cancer
1998,Gavin Wilson,male,4/1976,Kentucky,High School,Employed,1,married,Asia,2,0.0,Own flat,diabetes


### Variable continues 

!!! L'année de naissance minimum est de 1932 !!!

## k-Anonymity

### Test with sorting values

In [16]:
#pd.options.display.max_rows = None

In [17]:
data = data.sort_values(by=["gender", "dob", "education", "employment", "children", 
                "marital_status", "number_vehicles", "accommodation"])

data.head()


,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
520,Christian Nguyen,0,00,Arizona,0,0,none,0,Europe,none,0.0,0,nephrology
605,Jeremiah Soto,0,00,Iowa,0,0,none,0,Africa,none,0.0,0,interal mdeicine
723,Landon Nguyen,0,00,unknown state,0,0,none,0,Europe,none,0.0,0,gastro-enterolgy
434,Antonio Flores,0,00,Washington,0,0,none,0,Europe,one,0.0,0,oncology
1528,Logan Martin,0,00,West Virginia,0,0,none,0,America,one,0.0,0,interal mdeicine


In [18]:
"""k_min = math.inf
k=0


for i in range(len(data)-1) : 
    print(k)

    if(data["gender"].iloc[i] == data["gender"].iloc[i+1]) :
        k+=1
    else :#(data["gender"].iloc[i] != data["gender"].iloc[i+1]) :
        print(k)
        if(k<k_min and k>0):
            k_min = k
            k=0

#print(k_min)"""

gr = data[["gender", "dob"]].groupby(["gender", "dob"])


is_k = (gr.count() > 2)
    


In [19]:
gr[["gender", "dob"][0]].count() > 2



gender  dob
0       00     True
        30     True
        40     True
        50     True
        60     True
        70     True
        80     True
        90     True
1       00     True
        30     True
        40     True
        50     True
        60     True
        70     True
        80     True
        90     True
Name: gender, dtype: bool

In [20]:
data

,id,gender,dob,zipcode,education,employment,children,marital_status,ancestry,number_vehicles,commute_time,accommodation,disease
520,Christian Nguyen,0,00,Arizona,0,0,none,0,Europe,none,0.00,0,nephrology
605,Jeremiah Soto,0,00,Iowa,0,0,none,0,Africa,none,0.00,0,interal mdeicine
723,Landon Nguyen,0,00,unknown state,0,0,none,0,Europe,none,0.00,0,gastro-enterolgy
434,Antonio Flores,0,00,Washington,0,0,none,0,Europe,one,0.00,0,oncology
1528,Logan Martin,0,00,West Virginia,0,0,none,0,America,one,0.00,0,interal mdeicine
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,Valerie Pitts,1,90,Minnesota,1,1,one,1,Africa,one,0.43,1,gastro-enterolgy
561,Elizabeth Jenkins,1,90,unknown state,1,1,one,1,Europe,one,1.77,1,oncology
927,Josephine Macias,1,90,unknown state,1,1,one,1,Africa,one,1.10,1,nephrology
1009,Josephine White,1,90,unknown state,1,1,one,1,America,one,2.10,1,gastro-enterolgy


### Test with k_means

In [21]:
data = pd.get_dummies(data, columns=["ancestry", "number_vehicles", "children"])
data_cluster = data 
data_cluster.drop(columns=["id", "disease"], inplace=True)



In [22]:
kmeans = KMeans(n_clusters=10, init='k-means++', random_state=0).fit(data_cluster)

c = Counter(kmeans.labels_)

print(c.items())

ValueError: could not convert string to float: 'Arizona'

En suivant la méthode de k means, lorsque l'on choisi 10 centres (choisi arbitrairement), nous obtenons des groupes équilibrés. un groupe n'est pas composé de peu d'observation. En d'autres termes, nous arrivons à créer via la méthode de k_means 10 groupes dont tous les individus sont proches sur l'ensemble des dimensions de quasi-identifier, ce qui signifie qiue dans chaque groupe, il y a environ 200 individus proches, et donc indistinguable l'un de l'autre.

## Modification supplémentaire